# Laboratory work #4

Import all significant libraries for this project.

In [21]:
# Import TensorFlow & Keras Libraries
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Input, Reshape
from keras.layers import Conv2D, GlobalMaxPooling2D, Conv2DTranspose
from keras.layers import Dropout, BatchNormalization
from keras.layers import LeakyReLU
from keras.losses import CategoricalCrossentropy, BinaryCrossentropy
from keras.metrics import Mean
from keras.optimizers import Adam, RMSprop
from keras.datasets import cifar10
from keras import utils

# Import scikit-lean libraries
from sklearn.model_selection import train_test_split

# Import other libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pathlib

## Exercise 1 (GANs)

#### Downloading Dataset
Our dataset is CIFAR-10 (from keras.datasets import cifar10)

### Preprocessing Dataset

In [12]:
# Set constant hyperparameters
batch_size = 32
num_channels = 3
num_classes = 10
image_size = 32
latent_dim = 128

In [13]:
# Download data from dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# Concatenate the data
all_digits = np.concatenate([x_train, x_test])
all_labels = np.concatenate([y_train, y_test])

In [14]:
print(all_digits.shape)
print(all_labels.shape)

(60000, 32, 32, 3)
(60000, 1)


In [15]:
# Scale the pixel values to [0, 1] range, and a channel dimension to the images,
# and one-hot encode the labels.
all_digits = all_digits.astype("float32") / 255.
all_digits = np.reshape(all_digits, (-1, image_size, image_size, num_channels))
all_labels = utils.to_categorical(all_labels, num_classes)

In [16]:
# Create tf.data.Dataset
dataset = tf.data.Dataset.from_tensor_slices((all_digits, all_labels))
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

In [17]:
# View to shape of Dataset
print(f"Shape of training images: {all_digits.shape}")
print(f"Shape of training labels: {all_labels.shape}")

Shape of training images: (60000, 32, 32, 3)
Shape of training labels: (60000, 10)


In [18]:
# Calculating the number of input channel for the generator and discriminator
generator_in_channels = latent_dim + num_classes
discriminator_in_channels = num_channels + num_classes
print(generator_in_channels, discriminator_in_channels)

138 13


### Build model

In [ ]:
# Create the discriminator
inputs = Input(shape=(image_size, image_size, num_channels))
x = Conv2D(64, 3, strides=(2, 2), padding="same", activation=LeakyReLU(alpha=0.2))(inputs)
x = Conv2D(128, 3, strides=(2, 2), padding="same", activation=LeakyReLU(alpha=0.2))(x)
x = GlobalMaxPooling2D()(x)
outputs = Dense(1)(x)
# Build discriminator
discriminator = Model(inputs, outputs, name="discriminator")

In [ ]:
# Create the generator
inputs = Input(shape=(generator_in_channels))
# We want to generate 128 + num_classes coefficients to reshape into a
# 7x7x(128 + num_classes) map.
x = Dense(7 * 7 * generator_in_channels, activation=LeakyReLU(alpha=0.2))(inputs)
x = Reshape((7, 7, generator_in_channels))(x)
x = Conv2DTranspose(128, 4, strides=(2, 2), padding="same", activation=LeakyReLU(alpha=0.2))(x)
x = Conv2DTranspose(128, 4, strides=(2, 2), padding="same", activation=LeakyReLU(alpha=0.2))(x)
outputs = Conv2D(1, (7, 7), padding="same", activation="sigmoid")
# Build generator
generator = Model(inputs, outputs, name="generator")

In [ ]:
# Create a Conditional GAN model
class ConditionalGAN(Model):

    def __init__(self, discriminator, generator, latent_dim):
        """"""
        super(ConditionalGAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.gen_loss_tracker = Mean(name="generator_loss")
        self.disc_loss_tracker = Mean(name="discriminator_loss")

    @property
    def metrics(self):
        """"""
        return [self.gen_loss_tracker, self.disc_loss_tracker]

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        """"""
        super(ConditionalGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def train_step(self, data):
        """"""
        # Unpack the data
        real_images, one_hot_labels = data

        # Add dummy dimensions to the labels so that they can be concatenated with
        # the images. This is for the discriminator.
        image_one_hot_labels = one_hot_labels[:, :, None, None]
        image_one_hot_labels = tf.repeat(
            image_one_hot_labels, repeats=[image_size * image_size]
        )
        image_one_hot_labels = tf.reshape(
            image_one_hot_labels, (-1, image_size, image_size, num_classes)
        )

### Fit model

### Evaluate model

### Conclusion

## Exercise 2 (Machine translation)

#### Downloading Dataset
Our dataset is  (from [kaggle]())

### Preprocessing Dataset

### Build model

### Fit model

### Evaluate model

### Conclusion

## Exercise 3 (Transformers)

#### Downloading Dataset
Our dataset is  (from [kaggle]())

### Preprocessing Dataset

### Build model

### Fit model

### Evaluate model

### Conclusion